### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
21,41052972,122834,79085,51531,0,0,1074,0,2024-02-03 18:00,2024-02-03 18:00,0,17,5553,4,1
22,40853928,89208,79085,1345,13,65,1074,1,2024-02-04 21:58,2024-02-04 21:58,3,0,23998,6,1
23,41052973,79085,122834,51531,0,0,1074,2,2024-02-05 18:00,2024-02-05 18:00,1,0,1790,2,1
24,41054719,62584,79085,51530,0,0,1074,3,2024-02-06 11:30,2024-02-06 11:30,0,11,3276,6,1
25,40853918,123016,79085,1345,14,65,1074,4,2024-02-07 21:58,2024-02-07 21:58,2,0,7305,2,1
26,41052971,79085,121640,51530,0,0,1074,5,2024-02-08 22:00,2024-02-08 22:00,0,2,1751,4,1
27,41054458,33265,79085,3,0,0,1074,6,2024-02-09 19:21,2024-02-09 19:21,0,0,6118,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,122834,60533,30316,0,0,0,0,0,2x2,9,6,6,41052972
1,79085,9711,39928,0,48,1,0,0,4-3-3a,9,10,9,41052972
0,89208,32681,30537,0,7,2,0,0,junçao,10,10,10,40853928
1,79085,37286,39430,0,12,3,0,0,4-3-3a,8,9,10,40853928
0,79085,30956,40565,3,11,2,0,0,4-3-3a,7,9,9,41052973
1,122834,39127,29518,0,2,3,0,1,Sampaio GIGANTE!,8,7,9,41052973
0,62584,59424,29396,1,0,2,0,0,433,5,5,6,41054719
1,79085,10988,41016,0,51,0,0,0,4-3-3a,9,9,10,41054719
0,123016,37738,34015,0,6,1,0,0,Ricks,9,6,10,40853918
1,79085,32456,36179,1,6,2,0,0,4-3-3a,9,9,10,40853918


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1052

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1345

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,6276,14,32,10,2,2,38,15,32523038010011,1345
1,6295,14,32,10,2,2,34,13,32521034010012,1345
2,89481,14,30,9,3,2,31,11,30520031009005,1345
3,79085,14,24,7,3,4,18,12,24506018007007,1345
4,89208,14,22,6,4,4,27,22,22505027006009,1345
5,78588,14,17,5,2,7,24,20,17504024005001,1345
6,80120,14,17,3,8,3,14,11,17503014003010,1345
7,121640,14,16,4,4,6,19,20,16499019004002,1345
8,122818,14,15,4,3,7,25,18,15507025004008,1345
9,123016,14,14,4,2,8,13,31,14482013004004,1345


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

3

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()